# Введение в нейросети для обработки последовательностей

## Примеры задач

Подходящие форматы входных данных:
- **Текст**
- Музыка
- Видео
- Временные ряды
- ДНК
- Короче все что можно представить в виде последовательности, если немного потанцевать с бубном.

Формальнее: пусть $X \subset \mathbb{R}^n$ - пространство входных данных. Если мы можем выделить разумное направление из n доступных и дискретизировать элементы $X$ по нему, то можно юзать нейросети для обработки последовательностей

По большей части будем рассматривать тексты, т.е. заниматься NLP.

Примеры задач NLP:
- классификация документов (по темам, рубрикам, жанрам и так далее)
- определение спама
- определение частей речи
- исправление орфографических ошибок и опечаток
- поиск ключевых слов, синонимов / антонимов в тексте
- распознавание именованных сущностей (имен, названий географических объектов, дат, номеров телефонов, адресов)
- определение эмоциональной окраски текста (sentiment analysis)
- поиск релевантных документов по запросу, а также их ранжирование
- задача суммаризации (автоматическое составление краткого пересказа текста)
- автоматический перевод с одного языка на другой (машинный перевод)
- диалоговые системы и чат-боты
- вопросно-ответные системы (выбор ответа из нескольких предложенных вариантов или вопросы с открытым ответом)
- кроме того, к NLP также относят задачу распознавания речи (Automated Speech Recognition, ASR)

## Определения

Пусть $X \subset \mathbb{R}^k$ - пространство входных символов (например эмбеддинги слов), а $Y \subset \mathbb{R}^l$ - пространство выходных символов. Обозначим за $X_{seq}$ пространство всевозможных конечных последовательностей элементов из $X$. Аналогично введем $Y_{seq}$

**Определение:**
- Many-to-one модель для работы с последовательностя - отображение $F: X_{seq} \longrightarrow Y$
- One-to-many модель для работы с последовательностя - отображение $F: X \longrightarrow Y_{seq}$
- Несинхронизированная many-to-many модель для работы с последовательностя - отображение $F: X_{seq} \longrightarrow Y_{seq}$
- Синхронизированная many-to-many модель для работы с последовательностями - как несинхронизированная, но с условием что $\forall x \in X_{seq}$  $|x| = |F(x)|$

**Примеры:**

<img src="../media/sequence_models_example.png">

Разберемся сначала с более простыми Many-to-one и Sync many-to-many

## Эмбеддинги


Для того, чтобы работать с данными с помощью нейросетей для обработки последовательностей нужно собственно уметь превращать элементы входной/выходной последовательности в элементы $X$/$Y$. $X$/$Y$ в таком случае называются пространствами эмбеддингов.
Возможны так же варианты, когда вся входная/выходная последовательность целиком укладывается в один вектор уже в $X_{seq}$/$Y_{seq}$. Тогда уже $X_{seq}$/$Y_{seq}$ можно назвать пространствами эмбеддингов.

Разберем все это на примере NLP:

### Bag-of-Words

Пусть $T$ - пространство всевозможных текстов на английском языке. Построим эмбеддинг $E: T \longrightarrow X_{seq}$ таким образом:
- Выберем словарь известных слов
- Удалим из него наиболее часто встречающиеся и не несущие особого смысла слова (the, a, an, I, etc.)
- Таким образом у нас получится $D = {d_1, d_2, ..., d_n}$
- Переводим текст $t$ в вектор $(n_{d_1}, n_{d_2}, ..., n_{d_n})$, где $n_{d_i}$ - количество вхождений слова $d_i$ в тексте $t$

Возможны так же различные трюки, вроде того, что мы формируем словарь из N-грам, т.е. последовательностей из N слов

In [30]:
import numpy as np
import ahocorasick
from wordfreq import top_n_list
from typing import List

class BoW:
    def __init__(self, dictionary: List[str]):
        self.dictionary = np.array(dictionary)
        self.dictionary_fsm = ahocorasick.Automaton()
        
        for idx, key in enumerate(self.dictionary):
            self.dictionary_fsm.add_word(key, (idx, key))
        self.dictionary_fsm.make_automaton()

    def transform(self, text: str):
        text = text.lower()
        embedding = np.zeros(self.dictionary.shape[0])
        
        for  end_index, (idx, _) in self.dictionary_fsm.iter(text):
            start_index = end_index - len(word) + 1
            
            if (start_index == 0 or text[start_index - 1] == ' ') and (len(text) == end_index + 1 or text[end_index + 1] == ' '):
                embedding[idx] += 1
        
        return embedding

    def transform_readable(self, text: str):
        text = text.lower()
        result = {}
        
        for  end_index, (_, word) in self.dictionary_fsm.iter(text):
            start_index = end_index - len(word) + 1
            
            if (start_index == 0 or text[start_index - 1] == ' ') and (len(text) == end_index + 1 or text[end_index + 1] == ' '):
                if word not in result:
                    result[word] = 1
                else:
                    result[word] += 1
        
        return result
        
dictionary = top_n_list("en", n=10000)[20:]
bow = BoW(dictionary)
bow.transform_readable("William Shakespeare was an English playwright, poet and actor. He is widely regarded as the greatest writer in the English language and the world's pre-eminent dramatist. He is often called England's national poet and the \"Bard of Avon\" or simply \"the Bard\". His extant works, including collaborations, consist of some 39 plays, 154 sonnets, three long narrative poems and a few other verses, some of uncertain authorship. His plays have been translated into every major living language and are performed more often than those of any other playwright. Shakespeare remains arguably the most influential writer in the English language, and his works continue to be studied and reinterpreted.")

{np.str_('william'): 1,
 np.str_('shakespeare'): 2,
 np.str_('an'): 1,
 np.str_('english'): 3,
 np.str_('poet'): 2,
 np.str_('he'): 2,
 np.str_('widely'): 1,
 np.str_('regarded'): 1,
 np.str_('greatest'): 1,
 np.str_('writer'): 2,
 np.str_('language'): 2,
 np.str_("world's"): 1,
 np.str_('often'): 2,
 np.str_('called'): 1,
 np.str_('national'): 1,
 np.str_('or'): 1,
 np.str_('simply'): 1,
 np.str_('his'): 3,
 np.str_('including'): 1,
 np.str_('consist'): 1,
 np.str_('some'): 2,
 np.str_('three'): 1,
 np.str_('long'): 1,
 np.str_('narrative'): 1,
 np.str_('poems'): 1,
 np.str_('few'): 1,
 np.str_('other'): 2,
 np.str_('uncertain'): 1,
 np.str_('plays'): 1,
 np.str_('been'): 1,
 np.str_('translated'): 1,
 np.str_('into'): 1,
 np.str_('every'): 1,
 np.str_('major'): 1,
 np.str_('living'): 1,
 np.str_('performed'): 1,
 np.str_('more'): 1,
 np.str_('than'): 1,
 np.str_('those'): 1,
 np.str_('any'): 1,
 np.str_('remains'): 1,
 np.str_('arguably'): 1,
 np.str_('most'): 1,
 np.str_('influentia

### TF-IDF

Главная проблема bag-of-words в том, что он никак не учитывает контекст в котором мы работает. Эту проблему частично решает TF-IDF. Пусть у нас есть коллекция документов $C = {F_1, F_2, ..., F_k}$ - наш контекст.

Идея аналогичная BoW, но теперь в итоговом векторе эмбединга на $i$-том месте стоит не просто число вхождений слова в текст, а велечина $TF(d_i, t) \cdot IDF(d_i, C)$, где
- $TF(d_i, t) = \frac{n_{d_i}}{|t|}$ - частота вхождения $d_i$ в текст (term frquency)
- $IDF(d_i, C) = \log(\frac{|C|}{|C_{d_i}|})$, где $|C_{d_i}|$ - количество документов из нашего контекста, которые содержат слово $d_i$. Этот множитель штрафует компоненты, отвечающие слишком распространённым токенам, и повышает вес специфических для отдельных текстов (и, вероятно, информативных) слов. (inversed document frequiency)

In [4]:
class TFIDF:
    pass

### Word2Vec

Word2Vec развивает идею работы с контекстом. Де-факто это попытка построить эмбеддинг слова исходя из контекста его употребления, что логично и обосновано с точки зрения лингвистики.

Рассмотрим один из вариантов построения w2v - CBoW (Continuous bag-of-words).
Зафиксируем гиперпараметры:
- Словарь $D = {d_1, d_2, ..., d_n}$
- Размер контекста $l$ (например, если 1, то контекстом слова считаются одно слово до него и одно после)
- Размерность пространства эмбеддингов $k$

Для каждого слова $t$ заведем векторы $v_t, \omega_t \in \mathbb{R}^k$. $v_t$ - эмбеддинг слова, когда оно является центральным. $\omega_t$ - эмбеддинг слова, когда оно является частью контекста.

Таким образом у нас $2 \cdot n \cdot k$ параметров в модели.

На вход подается $C = {c_1, ..., c_{2*l}}$ - контекст ($l$ слов слева от центрального и столько же справа). Для каждого слова из словаря мы вычисляем $logits_d = (\sum_{i = 1}^{2*l} \omega_{c_i}, v_d)$. Затем с помощью $softmax(logits)$ вычисляем предсказанное распределение центральных слов.

В конце-концов считаем loss с помощью кросс-энтропии с истинным распределением и обучаем модель с помощью SGD

<img src="../media/CBOW.png"/>



## Рекуррентные нейронные сети

<img src="../media/RNN.png">

### Пример использования

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
class AirQualityRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='tanh')
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        out, _ = self.rnn(x)  # (batch_size, seq_len, hidden_size)
        out = out[:, -1, :]   # Take last output
        return self.linear(out)

In [ ]:
# Training setup for RNN
rnn_model = AirQualityRNN(input_size=train_data.shape[1], 
                         hidden_size=128, 
                         num_layers=2).to(device)
rnn_optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)

# Training loop for RNN
print("\nTraining Standard RNN:")
for epoch in range(NUM_EPOCHS):
    rnn_model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        rnn_optimizer.zero_grad()
        outputs = rnn_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        rnn_optimizer.step()
    
    # Validation
    rnn_model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            y_pred = rnn_model(X_test)
            test_loss += criterion(y_pred, y_test).item()
    
    print(f'Epoch {epoch+1:2} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss/len(test_loader):.4f}')

# RNN Evaluation
rnn_model.eval()
rnn_predictions, rnn_actuals = [], []
with torch.no_grad():
    for X_test, y_test in test_loader:
        X_test = X_test.to(device)
        y_pred = rnn_model(X_test).cpu().numpy().flatten()
        rnn_predictions.extend(y_pred)
        rnn_actuals.extend(y_test.numpy().flatten())

# Inverse scaling for RNN
rnn_predictions = np.array(rnn_predictions) * scaler.scale_[CO_GT_INDEX] + scaler.mean_[CO_GT_INDEX]
rnn_actuals = np.array(rnn_actuals) * scaler.scale_[CO_GT_INDEX] + scaler.mean_[CO_GT_INDEX]

In [ ]:
# Calculate RMSE for RNN
rnn_rmse = np.sqrt(mean_squared_error(rnn_actuals, rnn_predictions))
print(f'\nRNN Final RMSE: {rnn_rmse:.2f}')

## LSTM

## GRU

## Attention

## Self-attention

## Источники

https://education.yandex.ru/handbook/ml/article/nejroseti-dlya-raboty-s-posledovatelnostyami

https://lena-voita.github.io/nlp_course/word_embeddings.html

Word2Vec - https://arxiv.org/abs/1301.3781